In [ ]:
!pip install crossrefapi

In [1]:
from xml.dom.minidom import parse
import xml.dom.minidom
import glob

filenames = glob.glob('./dataset/metadata/*.xml')
doi = set()


In [2]:
for fp in filenames:
    # Open XML document using minidom parser
    DOMTree = xml.dom.minidom.parse(fp)
    collection = DOMTree.documentElement
    books = collection.getElementsByTagName("book-id")
    articles = collection.getElementsByTagName("article-id")
    
    # Ensure that publication IDs are DOI registered before collating together
    for book in books:
        if book.getAttribute("book-id-type") == "doi":
            doi.add(book.firstChild.data)
    for article in articles:
        if article.getAttribute("pub-id-type") == "doi":
            doi.add(article.firstChild.data)
print(len(doi))

3920


In [3]:
print(len(filenames))
print(len(doi))

24999
3920


In [4]:
import random
samp_doi = random.choices(list(doi), k=1000)
print(len(samp_doi))

1000


In [ ]:
from crossref.restful import Works
works = Works()
data = []
count = 0
for link in samp_doi:
    try:
        data.append(works.doi(link))
    except:
        print('issue with ', link)
        
    count += 1
    print('done with ', count, ' of ', len(samp_doi))

In [ ]:
links = []
for i in data:
    if i:
        links.append(i['URL'])
links

In [7]:
f = open("jstor_doi_unseen1000.txt", "w")
for link in links:
    f.write(link + "\n")
f.close()

In [8]:
len(links)

907

In [ ]:
styles = {'apa', 'harvard3', 'chicago-author-date', 'ieee', 'mla', 'american-chemical-society', 'acm-sig-proceedings'}
DATAPATH = './dataset/jstor/unseen1000'
for style in styles:
    fname = DATAPATH + 'jstor_'+ style
    counter = 0
    for link in links:
        !curl -LH "Accept: text/x-bibliography; style=$style" $link >> {fname}.txt
        counter += 1
        # print(style, counter, "of ", len(links), " completed" )